In [1]:
#clear all
%reset -f

#import packages
import numpy as np
import sys
import os
import pandas as pd
import mne
import matplotlib
import h5py
from sklearn.utils import resample
from mne_icalabel import label_components

root = 'F:/Documents/Science/MirRevAdaptEEG'
participants = list(range(0,32))
#specify which erp we are analyzing
erps = 'lrp'

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

In [2]:
#setting up path/ directory
#access trial type epoched data for each participant
def load_tfr_epochs(pp_num, root_dir, erp_path, task):

    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    filename = os.path.join(pp_directory, 'p%03d_%s-epo.fif' % (pp_num, task))

    epochs = mne.read_epochs(filename)

    return epochs, pp_directory

In [3]:
#set up parameters for time-frequency morlet convolution
def set_tfr_params(l_freq = 6, h_freq = 35, num_freq = 50, cycle = 6, samp_rate = 200):
    freqs = np.logspace(*np.log10([l_freq, h_freq]), num = num_freq)
    n_cycles = cycle
    sfreq = samp_rate
    
    return freqs, n_cycles, sfreq

In [4]:
#save tfr'd data
def output_tfr_data(root_dir, data, task, output):
    # Save the tfr'd data
    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/tfr_ana/%s' % (erps))
    out_fname = os.path.join(data_directory, '%s_%s-tfr.h5' % (task, output))
    mne.time_frequency.write_tfrs(out_fname, tfr=data, overwrite = True)

In [5]:
#load tfr'd data
def load_tfr_data(root_dir, task, output):
    # Save the tfr'd data
    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/tfr_ana/%s' % (erps))
    out_fname = os.path.join(data_directory, '%s_%s-tfr.h5' % (task, output))
    dat = mne.time_frequency.read_tfrs(out_fname)
    
    return dat

In [6]:
# save tfr plot
def save_tfr_plot(root_dir, fig, task, output):
    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'doc/fig/tfr_mne/%s' % (erps))
    out_fname = os.path.join(data_directory, '%s_%s-tfr.svg' % (task, output))
    fig.set_size_inches(8,6)
    fig.savefig(out_fname)
    matplotlib.pyplot.close(fig)

In [7]:
#save average tfr across participants for each condition
#morlet convolution for each participant, according to trial types - POWER

perturb_conds = ['small_large_aligned', 'smlerrors_rot', 'lrgerrors_rot', 'smlerrors_mir', 'lrgerrors_mir', 'smlerrors_rdm', 'lrgerrors_rdm']

for pcond in range(0, len(perturb_conds)):
    pp_power = []
    for pp in participants:
        data, ppdir = load_tfr_epochs(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond])
        freqs, n_cycles, sfreq = set_tfr_params()
        
        power = mne.time_frequency.tfr_morlet(
            data,
            freqs=freqs,
            n_cycles=n_cycles,
            use_fft=False,
            decim=1,
            n_jobs=None,
            return_itc = False,
            average = False
        )
        #average epochs within participant first, then across participants
        win_power = power.average()
        pp_power.append(win_power)
    #grand average of tfr across participants    
    alldat = mne.grand_average(pp_power, interpolate_bads=True, drop_bads=True)
    #save tfr object
    output_tfr_data(root_dir = root, data = alldat, task = perturb_conds[pcond], output = "power")

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\lrp\p000_small_large_aligned-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
48 matching events found
No baseline correction applied
0 projection items activated


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:   16.8s finished


Not setting metadata
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\lrp\p001_small_large_aligned-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
48 matching events found
No baseline correction applied
0 projection items activated


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:   16.7s finished


Not setting metadata
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p002\lrp\p002_small_large_aligned-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
48 matching events found
No baseline correction applied
0 projection items activated


...

Not setting metadata
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p030\lrp\p030_lrgerrors_rdm-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
42 matching events found
No baseline correction applied
0 projection items activated


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:   14.5s finished


Not setting metadata
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p031\lrp\p031_lrgerrors_rdm-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
42 matching events found
No baseline correction applied
0 projection items activated


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:   14.6s finished


Not setting metadata
Identifying common channels ...


In [8]:
#plot average tfr for each of perturbation conditions/ trial types
if erps == 'frn':
    baseline_t = [-1.5, -1.2]
elif erps == 'lrp':
    baseline_t = [-1.3, -1.0]
perturb_conds = ['small_large_aligned', 'smlerrors_rot', 'lrgerrors_rot', 'smlerrors_mir', 'lrgerrors_mir', 'smlerrors_rdm', 'lrgerrors_rdm']
channels = ['Fpz', 'AFz', #frontal (cognitive)
            'Fz', 'F3', 'F4', 'FCz', #medial frontal (motor preparatory)
            'Cz', 'C3', 'C4', #motor execution
            'Pz', 'P3', 'P4', #parietal areas
            'Oz', 'O1', 'O2'] #visual areas
colormap = 'coolwarm'
for pcond in range(0, len(perturb_conds)):
    data = load_tfr_data(root_dir=root, task = perturb_conds[pcond], output = "power")
    if erps == 'frn':
        fig = data[0].plot(picks = channels, baseline=(baseline_t[0], baseline_t[1]), mode="logratio", combine='mean', tmin=-1.5, tmax=1.5, title="%s" % (perturb_conds[pcond]), cmap = colormap)
    elif erps == 'lrp':
        fig = data[0].plot(picks = channels, baseline=(baseline_t[0], baseline_t[1]), mode="logratio", combine='mean', tmin=-1.5, tmax=1.5, title="%s" % (perturb_conds[pcond]), cmap = colormap)
        
    save_tfr_plot(root_dir=root, fig=fig[0], task = perturb_conds[pcond], output = "power")

Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/lrp\small_large_aligned_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/lrp\smlerrors_rot_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/lrp\lrgerrors_rot_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/lrp\smlerrors_mir_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/lrp\lrgerrors_mir_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/lrp\smlerrors_rdm_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/lrp\lrgerrors_rdm_power-tfr.h5 ...
Applying baseline correction (mode: logratio)


In [9]:
# subtract aligned from each perturbation condition (e.g. smlerrors_rot minus aligned)
perturb_conds = ['small_large_aligned', 'smlerrors_rot', 'lrgerrors_rot', 'smlerrors_mir', 'lrgerrors_mir', 'smlerrors_rdm', 'lrgerrors_rdm']

if erps == 'frn':
    baseline_t = [-1.5, -1.2]
elif erps == 'lrp':
    baseline_t = [-1.3, -1.0]

smlerrors_rot = []
lrgerrors_rot = []
smlerrors_mir = []
lrgerrors_mir = []
smlerrors_rdm = []
lrgerrors_rdm = []
    
for pp in participants:
    for pcond in range(0, len(perturb_conds)):
        data, ppdir = load_tfr_epochs(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond])
        freqs, n_cycles, sfreq = set_tfr_params()
        
        power = mne.time_frequency.tfr_morlet(
            data,
            freqs=freqs,
            n_cycles=n_cycles,
            use_fft=False,
            decim=1,
            n_jobs=None,
            return_itc = False,
            average = False
        )
        
        if pcond == 0:
            aln_power = power.average()
            aln_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
        elif pcond == 1:
            smlerrors_rot_power = power.average()
            smlerrors_rot_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
        elif pcond == 2:
            lrgerrors_rot_power = power.average()
            lrgerrors_rot_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
        elif pcond == 3:
            smlerrors_mir_power = power.average()
            smlerrors_mir_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
        elif pcond == 4:
            lrgerrors_mir_power = power.average()
            lrgerrors_mir_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
        elif pcond == 5:
            smlerrors_rdm_power = power.average()
            smlerrors_rdm_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
        elif pcond == 6:
            lrgerrors_rdm_power = power.average()
            lrgerrors_rdm_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
            
    smlerrors_rot_contrast = smlerrors_rot_power - aln_power
    lrgerrors_rot_contrast = lrgerrors_rot_power - aln_power
    smlerrors_mir_contrast = smlerrors_mir_power - aln_power
    lrgerrors_mir_contrast = lrgerrors_mir_power - aln_power
    smlerrors_rdm_contrast = smlerrors_rdm_power - aln_power
    lrgerrors_rdm_contrast = lrgerrors_rdm_power - aln_power
    
    smlerrors_rot.append(smlerrors_rot_contrast)
    lrgerrors_rot.append(lrgerrors_rot_contrast)
    smlerrors_mir.append(smlerrors_mir_contrast)
    lrgerrors_mir.append(lrgerrors_mir_contrast)
    smlerrors_rdm.append(smlerrors_rdm_contrast)
    lrgerrors_rdm.append(lrgerrors_rdm_contrast)
    

#grand average of tfr contrasts across participants    
alldat = mne.grand_average(smlerrors_rot, interpolate_bads=True, drop_bads=True)
#save tfr object
output_tfr_data(root_dir = root, data = alldat, task = 'smlerrors_rot_contrast', output = "power")
 
alldat = mne.grand_average(lrgerrors_rot, interpolate_bads=True, drop_bads=True)
output_tfr_data(root_dir = root, data = alldat, task = 'lrgerrors_rot_contrast', output = "power")

alldat = mne.grand_average(smlerrors_mir, interpolate_bads=True, drop_bads=True)
output_tfr_data(root_dir = root, data = alldat, task = 'smlerrors_mir_contrast', output = "power")

alldat = mne.grand_average(lrgerrors_mir, interpolate_bads=True, drop_bads=True)
output_tfr_data(root_dir = root, data = alldat, task = 'lrgerrors_mir_contrast', output = "power")

alldat = mne.grand_average(smlerrors_rdm, interpolate_bads=True, drop_bads=True)
output_tfr_data(root_dir = root, data = alldat, task = 'smlerrors_rdm_contrast', output = "power")

alldat = mne.grand_average(lrgerrors_rdm, interpolate_bads=True, drop_bads=True)
output_tfr_data(root_dir = root, data = alldat, task = 'lrgerrors_rdm_contrast', output = "power")



Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\lrp\p000_small_large_aligned-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
48 matching events found
No baseline correction applied
0 projection items activated


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:   16.8s finished


Not setting metadata
Applying baseline correction (mode: logratio)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\lrp\p000_smlerrors_rot-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
36 matching events found
No baseline correction applied
0 projection items activated


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.7s remaining:    0.0s


Not setting metadata


...

Applying baseline correction (mode: logratio)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p031\lrp\p031_smlerrors_rdm-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
42 matching events found
No baseline correction applied
0 projection items activated


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:   14.6s finished


Not setting metadata
Applying baseline correction (mode: logratio)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p031\lrp\p031_lrgerrors_rdm-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
42 matching events found
No baseline correction applied
0 projection items activated


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:   14.7s finished


Not setting metadata
Applying baseline correction (mode: logratio)
Identifying common channels ...
Identifying common channels ...
Identifying common channels ...
Identifying common channels ...
Identifying common channels ...
Identifying common channels ...


In [10]:
#plot ALIGNED SUBTRACTED average tfr  contrasts for each of perturbation conditions/ trial types
# no baselining needed, since this was already done before subtraction
perturb_conds = ['smlerrors_rot_contrast', 'lrgerrors_rot_contrast', 'smlerrors_mir_contrast', 'lrgerrors_mir_contrast', 'smlerrors_rdm_contrast', 'lrgerrors_rdm_contrast']

channels = ['Fpz', 'AFz', #frontal (cognitive)
            'Fz', 'F3', 'F4', 'FCz', #medial frontal (motor preparatory)
            'Cz', 'C3', 'C4', #motor execution
            'Pz', 'P3', 'P4', #parietal areas
            'Oz', 'O1', 'O2'] #visual areas
colormap = 'coolwarm'
for pcond in range(0, len(perturb_conds)):
    data = load_tfr_data(root_dir=root, task = perturb_conds[pcond], output = "power")
    if erps == 'frn':
        fig = data[0].plot(picks = channels, combine='mean', tmin=-1.5, tmax=1.5, title="%s" % (perturb_conds[pcond]), cmap = colormap)
    elif erps == 'lrp':
        fig = data[0].plot(picks = channels, combine='mean', tmin=-1.5, tmax=1.5, title="%s" % (perturb_conds[pcond]), cmap = colormap)
        
    save_tfr_plot(root_dir=root, fig=fig[0], task = perturb_conds[pcond], output = "power")

Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/lrp\smlerrors_rot_contrast_power-tfr.h5 ...
No baseline correction applied
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/lrp\lrgerrors_rot_contrast_power-tfr.h5 ...
No baseline correction applied
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/lrp\smlerrors_mir_contrast_power-tfr.h5 ...
No baseline correction applied
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/lrp\lrgerrors_mir_contrast_power-tfr.h5 ...
No baseline correction applied
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/lrp\smlerrors_rdm_contrast_power-tfr.h5 ...
No baseline correction applied
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/lrp\lrgerrors_rdm_contrast_power-tfr.h5 ...
No baseline correction applied


In [11]:
# subtract SMALL from LARGE for each perturbation type (e.g. lrgerrors_rot minus smlerrors_rot)
# subtract baseline aligned for each participant, then subtract small from large
perturb_conds = ['small_large_aligned', 'smlerrors_rot', 'lrgerrors_rot', 'smlerrors_mir', 'lrgerrors_mir', 'smlerrors_rdm', 'lrgerrors_rdm']

if erps == 'frn':
    baseline_t = [-1.5, -1.2]
elif erps == 'lrp':
    baseline_t = [-1.3, -1.0]

rot = []
mir = []
rdm = []

    
for pp in participants:
    for pcond in range(0, len(perturb_conds)):
        data, ppdir = load_tfr_epochs(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond])
        freqs, n_cycles, sfreq = set_tfr_params()
        
        power = mne.time_frequency.tfr_morlet(
            data,
            freqs=freqs,
            n_cycles=n_cycles,
            use_fft=False,
            decim=1,
            n_jobs=None,
            return_itc = False,
            average = False
        )
        
        if pcond == 0:
            aln_power = power.average()
            aln_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
        elif pcond == 1:
            smlerrors_rot_power = power.average()
            smlerrors_rot_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
        elif pcond == 2:
            lrgerrors_rot_power = power.average()
            lrgerrors_rot_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
        elif pcond == 3:
            smlerrors_mir_power = power.average()
            smlerrors_mir_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
        elif pcond == 4:
            lrgerrors_mir_power = power.average()
            lrgerrors_mir_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
        elif pcond == 5:
            smlerrors_rdm_power = power.average()
            smlerrors_rdm_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
        elif pcond == 6:
            lrgerrors_rdm_power = power.average()
            lrgerrors_rdm_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
            
    smlerrors_rot_contrast = smlerrors_rot_power - aln_power
    lrgerrors_rot_contrast = lrgerrors_rot_power - aln_power
    smlerrors_mir_contrast = smlerrors_mir_power - aln_power
    lrgerrors_mir_contrast = lrgerrors_mir_power - aln_power
    smlerrors_rdm_contrast = smlerrors_rdm_power - aln_power
    lrgerrors_rdm_contrast = lrgerrors_rdm_power - aln_power
    
    rot_contrast = lrgerrors_rot_contrast - smlerrors_rot_contrast
    mir_contrast = lrgerrors_mir_contrast - smlerrors_mir_contrast
    rdm_contrast = lrgerrors_rdm_contrast - smlerrors_rdm_contrast
    #print(lrgerrors_rot_contrast, smlerrors_rot_contrast, rot_contrast)
    rot.append(rot_contrast)
    mir.append(mir_contrast)
    rdm.append(rdm_contrast)
    

#grand average of tfr contrasts across participants    
alldat = mne.grand_average(rot, interpolate_bads=True, drop_bads=True)
#save tfr object
output_tfr_data(root_dir = root, data = alldat, task = 'SL_rot', output = "power")
 
alldat = mne.grand_average(mir, interpolate_bads=True, drop_bads=True)
output_tfr_data(root_dir = root, data = alldat, task = 'SL_mir', output = "power")

alldat = mne.grand_average(rdm, interpolate_bads=True, drop_bads=True)
output_tfr_data(root_dir = root, data = alldat, task = 'SL_rdm', output = "power")



Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\lrp\p000_small_large_aligned-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
48 matching events found
No baseline correction applied
0 projection items activated


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:   16.8s finished


Not setting metadata
Applying baseline correction (mode: logratio)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\lrp\p000_smlerrors_rot-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
36 matching events found
No baseline correction applied
0 projection items activated


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:   13.5s finished


Not setting metadata
Applying baseline correction (mode: logratio)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\lrp\p000_lrgerrors_rot-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
36 matching events found
No baseline correction applied
0 projection items activated


...

Applying baseline correction (mode: logratio)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p031\lrp\p031_smlerrors_rdm-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
42 matching events found
No baseline correction applied
0 projection items activated


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:   14.6s finished


Not setting metadata
Applying baseline correction (mode: logratio)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p031\lrp\p031_lrgerrors_rdm-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
42 matching events found
No baseline correction applied
0 projection items activated


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:   14.6s finished


Not setting metadata
Applying baseline correction (mode: logratio)
Identifying common channels ...
Identifying common channels ...
Identifying common channels ...


In [12]:
#plot SMALL and LARGE SUBTRACTED average tfr  contrasts for each of perturbation conditions/ trial types
# no baselining needed, since this was already done before subtraction
perturb_conds = ['SL_rot', 'SL_mir', 'SL_rdm']
channels = ['Fpz', 'AFz', #frontal (cognitive)
            'Fz', 'F3', 'F4', 'FCz', #medial frontal (motor preparatory)
            'Cz', 'C3', 'C4', #motor execution
            'Pz', 'P3', 'P4', #parietal areas
            'Oz', 'O1', 'O2'] #visual areas
colormap = 'coolwarm'
for pcond in range(0, len(perturb_conds)):
    data = load_tfr_data(root_dir=root, task = perturb_conds[pcond], output = "power")
    if erps == 'frn':
        fig = data[0].plot(picks = channels, combine='mean', tmin=-1.5, tmax=1.5, title="%s" % (perturb_conds[pcond]), cmap = colormap)
    elif erps == 'lrp':
        fig = data[0].plot(picks = channels, combine='mean', tmin=-1.5, tmax=1.5, title="%s" % (perturb_conds[pcond]), cmap = colormap)
    
    save_tfr_plot(root_dir=root, fig=fig[0], task = perturb_conds[pcond], output = "power")

Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/lrp\SL_rot_power-tfr.h5 ...
No baseline correction applied
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/lrp\SL_mir_power-tfr.h5 ...
No baseline correction applied
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/lrp\SL_rdm_power-tfr.h5 ...
No baseline correction applied


In [13]:
# convert SMALL and LARGE SUBTRACTED TFR to CSV
perturb_conds = ['SL_rot', 'SL_mir', 'SL_rdm']

for pcond in range(0, len(perturb_conds)):
    data = load_tfr_data(root_dir=root, task = perturb_conds[pcond], output = "power")
    df = data[0].to_data_frame()
    df = df[['time', 'freq', 'Fpz', 'AFz', 'Fz', 'F3', 'F4', 'FCz', 'Cz', 'C3', 'C4', 'Pz', 'P3', 'P4', 'Oz', 'O1', 'O2']]
    df_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', '%s_power_%s.csv' % (perturb_conds[pcond], erps))
    df.to_csv(df_filename)
    

Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/lrp\SL_rot_power-tfr.h5 ...
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/lrp\SL_mir_power-tfr.h5 ...
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/lrp\SL_rdm_power-tfr.h5 ...


In [14]:
# perturb_conds = ['small_large_aligned', 'smlerrors_rot', 'lrgerrors_rot', 'smlerrors_mir', 'lrgerrors_mir', 'smlerrors_rdm', 'lrgerrors_rdm']

# for pcond in range(0, len(perturb_conds)):
#     pp_power = []
#     pp=0
#     data, ppdir = load_tfr_epochs(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond])
# #     data.compute_psd(fmin=2.0, fmax=100).plot(average=True, picks="data", exclude="bads")
#     data.compute_psd().plot_topomap(ch_type="eeg", normalize=False, contours=0)